# Chapter 18: Kubernetes Security

Having established sophisticated workload patterns in Chapter 17, we now implement defense-in-depth security controls to protect these resources. Kubernetes security operates at multiple layers: identity and access management through RBAC, runtime protection via Pod Security Standards, network micro-segmentation, supply chain security with admission controllers, and audit logging for compliance. This chapter implements the zero-trust principle for container orchestration—never trust, always verify, and enforce least privilege at every boundary.

Security in Kubernetes requires balancing operational flexibility with restrictive controls. Too permissive configurations expose attack surfaces; too restrictive policies break legitimate workflows. The practices here align with CIS Kubernetes Benchmarks, NIST SP 800-190 (Container Security Guide), and OWASP Container Security Verification Standard.

## 18.1 RBAC (Role-Based Access Control)

RBAC regulates access to Kubernetes API resources based on the roles of individual users or service accounts within an organization. It replaces the deprecated Attribute-Based Access Control (ABAC) with a declarative, version-controllable permission system.

### RBAC Architecture

RBAC consists of four primary resources:
- **Role/ClusterRole**: Define permissions (what actions on which resources)
- **RoleBinding/ClusterRoleBinding**: Assign roles to subjects (who gets the permissions)

**Scope Distinction:**
- **Role**: Namespaced; permissions apply within one namespace
- **ClusterRole**: Cluster-wide; permissions apply across all namespaces or non-namespaced resources (nodes, PersistentVolumes)

### Principle of Least Privilege Implementation

**Default Deny Posture:**
Kubernetes starts with no permissions. Every API request denied unless explicitly allowed through RBAC.

**User vs Service Account:**
- **Users**: External identities (humans, CI/CD pipelines) authenticated via certificates, OIDC, or webhooks
- **ServiceAccounts**: In-cluster identities for Pods and controllers

### Role Definitions

**Namespace-Scoped Role:**
```yaml
apiVersion: rbac.authorization.k8s.io/v1
kind: Role
metadata:
  namespace: production
  name: deployment-manager
rules:
- apiGroups: ["apps"]
  resources: ["deployments"]
  verbs: ["get", "list", "watch", "create", "update", "patch", "delete"]
  resourceNames: []  # Empty = all deployments in namespace
  
- apiGroups: [""]
  resources: ["pods"]
  verbs: ["get", "list", "watch"]
  
- apiGroups: [""]
  resources: ["pods/log"]
  verbs: ["get", "list"]
  
- apiGroups: [""]
  resources: ["pods/exec"]
  verbs: ["create"]  # Allow kubectl exec
  
- apiGroups: [""]
  resources: ["configmaps"]
  verbs: ["get", "list"]
  resourceNames: ["app-config", "feature-flags"]  # Specific configmaps only
```

**Cluster-Wide Role:**
```yaml
apiVersion: rbac.authorization.k8s.io/v1
kind: ClusterRole
metadata:
  name: node-reader
rules:
- apiGroups: [""]
  resources: ["nodes"]
  verbs: ["get", "list", "watch"]
  
- apiGroups: [""]
  resources: ["nodes/status"]
  verbs: ["get"]
  
- apiGroups: ["metrics.k8s.io"]
  resources: ["nodes"]
  verbs: ["get", "list"]
```

### Binding Subjects to Roles

**RoleBinding (Namespaced):**
```yaml
apiVersion: rbac.authorization.k8s.io/v1
kind: RoleBinding
metadata:
  name: deployment-manager-binding
  namespace: production
subjects:
- kind: User
  name: "alice@company.com"
  apiGroup: rbac.authorization.k8s.io
- kind: Group
  name: "developers"
  apiGroup: rbac.authorization.k8s.io
- kind: ServiceAccount
  name: cicd-pipeline
  namespace: cicd
roleRef:
  kind: Role
  name: deployment-manager
  apiGroup: rbac.authorization.k8s.io
```

**ClusterRoleBinding:**
```yaml
apiVersion: rbac.authorization.k8s.io/v1
kind: ClusterRoleBinding
metadata:
  name: node-reader-binding
subjects:
- kind: Group
  name: "monitoring-team"
  apiGroup: rbac.authorization.k8s.io
roleRef:
  kind: ClusterRole
  name: node-reader
  apiGroup: rbac.authorization.k8s.io
```

### Common RBAC Patterns

**Namespace Admin:**
```yaml
apiVersion: rbac.authorization.k8s.io/v1
kind: Role
metadata:
  name: namespace-admin
rules:
- apiGroups: ["*"]
  resources: ["*"]
  verbs: ["*"]
---
apiVersion: rbac.authorization.k8s.io/v1
kind: RoleBinding
metadata:
  name: namespace-admin-binding
subjects:
- kind: Group
  name: "production-team"
  apiGroup: rbac.authorization.k8s.io
```

**Read-Only Access:**
```yaml
apiVersion: rbac.authorization.k8s.io/v1
kind: ClusterRole
metadata:
  name: view-only
rules:
- apiGroups: ["*"]
  resources: ["*"]
  verbs: ["get", "list", "watch"]
- apiGroups: [""]
  resources: ["secrets"]
  verbs: []  # Explicitly deny secrets access
```

**CI/CD Automation:**
```yaml
apiVersion: rbac.authorization.k8s.io/v1
kind: Role
metadata:
  namespace: production
  name: deployer
rules:
- apiGroups: ["apps"]
  resources: ["deployments"]
  verbs: ["get", "update", "patch"]
  resourceNames: ["web-app", "api-service"]
  
- apiGroups: [""]
  resources: ["pods"]
  verbs: ["get", "list"]
  
- apiGroups: [""]
  resources: ["pods/log"]
  verbs: ["get", "list"]
```

### Verifying RBAC

**Impersonation Testing:**
```bash
# Test if user can perform action
kubectl auth can-i create deployments --namespace production --as alice@company.com

# Check specific resource
kubectl auth can-i delete pods/nginx-7d8bc9c5b8-abc12 --namespace default --as bob

# List all permissions
kubectl auth can-i --list --namespace production --as alice@company.com
```

**Debugging Denials:**
```bash
# Check audit logs for RBAC denials
kubectl logs -n kube-system kube-apiserver-master | grep "RBAC DENY"
```

## 18.2 Service Accounts

ServiceAccounts provide identity for in-cluster processes. Every Pod runs with a ServiceAccount; if unspecified, Kubernetes assigns the `default` ServiceAccount in the namespace.

### Default ServiceAccount Risks

The default ServiceAccount has no RBAC permissions but possesses a token that can authenticate to the API server. This creates attack surface if compromised.

**Security Hardening:**
```yaml
apiVersion: v1
kind: ServiceAccount
metadata:
  name: secure-app
  namespace: production
automountServiceAccountToken: false  # Prevent token mounting unless explicitly needed
```

### Token Projection (Modern Approach)

Kubernetes 1.20+ introduces ServiceAccount token projection using VolumeProjection, providing time-bound, audience-scoped tokens rather than long-lived secrets.

```yaml
apiVersion: v1
kind: ServiceAccount
metadata:
  name: api-service
  namespace: production
---
apiVersion: apps/v1
kind: Deployment
metadata:
  name: api-service
spec:
  template:
    spec:
      serviceAccountName: api-service
      automountServiceAccountToken: false  # Disable default token
      containers:
      - name: api
        image: api:latest
        volumeMounts:
        - name: service-account-token
          mountPath: /var/run/secrets/tokens
          readOnly: true
      volumes:
      - name: service-account-token
        projected:
          sources:
          - serviceAccountToken:
              path: api-token
              expirationSeconds: 3600  # 1 hour
              audience: api-server
              - serviceAccountToken:
              path: vault-token
              expirationSeconds: 7200
              audience: vault
```

### RBAC for ServiceAccounts

**Least Privilege for Applications:**
```yaml
apiVersion: rbac.authorization.k8s.io/v1
kind: Role
metadata:
  namespace: production
  name: config-reader
rules:
- apiGroups: [""]
  resources: ["configmaps"]
  verbs: ["get"]
  resourceNames: ["app-config"]
---
apiVersion: rbac.authorization.k8s.io/v1
kind: RoleBinding
metadata:
  name: config-reader-binding
  namespace: production
subjects:
- kind: ServiceAccount
  name: api-service
  namespace: production
roleRef:
  kind: Role
  name: config-reader
  apiGroup: rbac.authorization.k8s.io
```

### Securing ServiceAccount Tokens

**Disable Secret Creation:**
```yaml
apiVersion: v1
kind: ServiceAccount
metadata:
  name: no-secrets
secrets: []  # Prevent creation of long-lived token secrets
```

**Token Request API:**
Applications should use TokenRequest API for short-lived tokens rather than reading static secrets:

```bash
# Request token via API (from within Pod)
curl -X POST "https://kubernetes.default.svc/api/v1/namespaces/production/serviceaccounts/api-service/token" \
  -H "Authorization: Bearer $(cat /var/run/secrets/kubernetes.io/serviceaccount/token)" \
  -H "Content-Type: application/json" \
  -d '{"kind":"TokenRequest","apiVersion":"authentication.k8s.io/v1","spec":{"audiences":["api-server"]}}'
```

## 18.3 Pod Security Standards

Pod Security Standards (PSS) replaced the deprecated PodSecurityPolicies (PSP) as the built-in policy enforcement mechanism. PSS defines three security levels implemented via the built-in Pod Security Admission controller (Kubernetes 1.23+, enabled by default 1.25+).

### Security Levels

**Privileged:**
Unrestricted policy allowing full access. Dangerous; use only for system-level workloads.

**Baseline:**
Prevents known privilege escalations while permitting common application configurations. Suitable for most applications.

**Restricted:**
Heavily restricted policy following current Pod hardening best practices. May break legacy applications.

### Enforcement Modes

- **enforce**: Violations cause Pod rejection
- **audit**: Violations trigger audit log entries (Pod admitted)
- **warn**: Violations trigger user-facing warnings (Pod admitted)

### Namespace Labeling

Apply standards via namespace labels:

```yaml
apiVersion: v1
kind: Namespace
metadata:
  name: production
  labels:
    pod-security.kubernetes.io/enforce: restricted
    pod-security.kubernetes.io/enforce-version: latest
    pod-security.kubernetes.io/audit: restricted
    pod-security.kubernetes.io/audit-version: latest
    pod-security.kubernetes.io/warn: restricted
    pod-security.kubernetes.io/warn-version: latest
```

**Exemptions:**
```yaml
apiVersion: v1
kind: Namespace
metadata:
  name: monitoring
  labels:
    pod-security.kubernetes.io/enforce: baseline
    pod-security.kubernetes.io/enforce-version: latest
```

### Policy Violations

**Restricted Policy Requirements:**
- Must run as non-root (runAsNonRoot: true or runAsUser != 0)
- Must drop all capabilities (ALL)
- Must restrict seccomp profiles (RuntimeDefault or Localhost)
- Must prevent privilege escalation (allowPrivilegeEscalation: false)
- Must restrict volume types (no hostPath, etc.)
- Must set resource limits (memory/cpu)
- Must use specific seccomp profiles

**Example Compliant Pod:**
```yaml
apiVersion: v1
kind: Pod
metadata:
  name: secure-app
  namespace: production
spec:
  securityContext:
    runAsNonRoot: true
    seccompProfile:
      type: RuntimeDefault
  containers:
  - name: app
    image: myapp:latest
    securityContext:
      allowPrivilegeEscalation: false
      capabilities:
        drop:
        - ALL
      readOnlyRootFilesystem: true
      runAsUser: 1000
      runAsGroup: 1000
      seccompProfile:
        type: RuntimeDefault
    resources:
      limits:
        memory: "512Mi"
        cpu: "500m"
      requests:
        memory: "128Mi"
        cpu: "100m"
    volumeMounts:
    - name: tmp
      mountPath: /tmp
    - name: cache
      mountPath: /cache
  volumes:
  - name: tmp
    emptyDir: {}
  - name: cache
    emptyDir: {}
```

### Migration from PodSecurityPolicy

If upgrading from PSPs:
1. Audit current PSPs using `kubectl get psp`
2. Map PSPs to PSS levels (Privileged, Baseline, Restricted)
3. Apply namespace labels corresponding to required level
4. Remove PSPs after verification (PSPs removed in Kubernetes 1.25)

## 18.4 Network Policies (Security Focus)

While Chapter 15 covered network implementation, the security perspective emphasizes default-deny posture and micro-segmentation.

### Zero-Trust Networking

**Default Deny All Ingress:**
```yaml
apiVersion: networking.k8s.io/v1
kind: NetworkPolicy
metadata:
  name: default-deny-ingress
  namespace: production
spec:
  podSelector: {}
  policyTypes:
  - Ingress
```

**Default Deny All Egress:**
```yaml
apiVersion: networking.k8s.io/v1
kind: NetworkPolicy
metadata:
  name: default-deny-egress
  namespace: production
spec:
  podSelector: {}
  policyTypes:
  - Egress
```

**Allow DNS Resolution:**
Required even with default deny:
```yaml
apiVersion: networking.k8s.io/v1
kind: NetworkPolicy
metadata:
  name: allow-dns
spec:
  podSelector: {}
  policyTypes:
  - Egress
  egress:
  - to:
    - namespaceSelector:
        matchLabels:
          name: kube-system
      podSelector:
        matchLabels:
          k8s-app: kube-dns
    ports:
    - protocol: UDP
      port: 53
    - protocol: TCP
      port: 53
```

## 18.5 Secrets Management

Chapter 14 introduced Kubernetes Secrets; here we address enterprise-grade secret lifecycle management integrating external secret management systems.

### External Secrets Operator (ESO)

ESO synchronizes secrets from external providers (AWS Secrets Manager, Azure Key Vault, GCP Secret Manager, HashiCorp Vault) into Kubernetes Secrets.

**Installation:**
```bash
helm repo add external-secrets https://charts.external-secrets.io
helm install external-secrets external-secrets/external-secrets \
  --namespace external-secrets \
  --create-namespace
```

**SecretStore (Cluster-scoped or Namespaced):**
```yaml
apiVersion: external-secrets.io/v1beta1
kind: ClusterSecretStore
metadata:
  name: aws-secrets-manager
spec:
  provider:
    aws:
      service: SecretsManager
      region: us-east-1
      auth:
        jwt:
          serviceAccountRef:
            name: external-secrets-sa
            namespace: external-secrets
```

**ExternalSecret Resource:**
```yaml
apiVersion: external-secrets.io/v1beta1
kind: ExternalSecret
metadata:
  name: db-credentials
  namespace: production
spec:
  refreshInterval: 1h
  secretStoreRef:
    kind: ClusterSecretStore
    name: aws-secrets-manager
  target:
    name: db-credentials
    creationPolicy: Owner
    template:
      type: Opaque
      data:
        connection-string: "postgresql://{{ .username }}:{{ .password }}@db:5432/app"
  data:
  - secretKey: username
    remoteRef:
      key: production/db-credentials
      property: username
  - secretKey: password
    remoteRef:
      key: production/db-credentials
      property: password
```

### Vault Agent Injector

HashiCorp Vault integration injects secrets directly into application containers without persisting in Kubernetes Secrets.

**Annotation-based Injection:**
```yaml
apiVersion: apps/v1
kind: Deployment
metadata:
  name: secure-app
spec:
  template:
    metadata:
      annotations:
        vault.hashicorp.com/agent-inject: "true"
        vault.hashicorp.com/role: "app-role"
        vault.hashicorp.com/agent-inject-secret-db-creds: "database/creds/app"
        vault.hashicorp.com/agent-inject-template-db-creds: |
          {{ with secret "database/creds/app" -}}
          export DB_USER="{{ .Data.username }}"
          export DB_PASS="{{ .Data.password }}"
          {{- end }}
    spec:
      serviceAccountName: secure-app
      containers:
      - name: app
        image: app:latest
        command: ["/bin/sh", "-c"]
        args: ["source /vault/secrets/db-creds && ./start-app"]
```

**Benefits:**
- Secrets never written to etcd
- Dynamic short-lived credentials
- Automatic rotation
- Audit trails in Vault

### Sealed Secrets

For GitOps workflows, encrypt secrets for safe storage in Git:

```bash
# Install kubeseal CLI
kubeseal --fetch-cert > pub-cert.pem

# Create secret and seal it
kubectl create secret generic mysecret --dry-run=client --from-literal=foo=bar -o yaml | \
  kubeseal --cert pub-cert.pem -o yaml > mysealedsecret.yaml

# Commit mysealedsecret.yaml to Git
kubectl apply -f mysealedsecret.yaml  # Only cluster can decrypt
```

## 18.6 Image Admission Controllers

Prevent deployment of vulnerable or unauthorized images using admission controllers that intercept API requests before persistence.

### OPA Gatekeeper (Open Policy Agent)

Policy-as-code enforcement using Rego language.

**Installation:**
```bash
helm repo add gatekeeper https://open-policy-agent.github.io/gatekeeper/charts
helm install gatekeeper gatekeeper/gatekeeper --namespace gatekeeper --create-namespace
```

**ConstraintTemplate:**
```yaml
apiVersion: templates.gatekeeper.sh/v1
kind: ConstraintTemplate
metadata:
  name: k8srequiredlabels
spec:
  crd:
    spec:
      names:
        kind: K8sRequiredLabels
      validation:
        openAPIV3Schema:
          properties:
            labels:
              type: array
              items:
                type: string
  targets:
    - target: admission.k8s.gatekeeper.sh
      rego: |
        package k8srequiredlabels
        violation[{"msg": msg}] {
          provided := {label | input.review.object.metadata.labels[label]}
          required := {label | label := input.parameters.labels[_]}
          missing := required - provided
          count(missing) > 0
          msg := sprintf("Missing required labels: %v", [missing])
        }
```

**Constraint:**
```yaml
apiVersion: constraints.gatekeeper.sh/v1beta1
kind: K8sRequiredLabels
metadata:
  name: require-cost-center
spec:
  match:
    kinds:
      - apiGroups: [""]
        kinds: ["Namespace"]
  parameters:
    labels: ["cost-center", "owner"]
```

**Image Security Policy:**
```yaml
apiVersion: templates.gatekeeper.sh/v1
kind: ConstraintTemplate
metadata:
  name: k8strustedimages
spec:
  targets:
    - target: admission.k8s.gatekeeper.sh
      rego: |
        package k8strustedimages
        violation[{"msg": msg}] {
          image := input.review.object.spec.containers[_].image
          not startswith(image, "company.registry.com/")
          msg := sprintf("Untrusted image: %s", [image])
        }
```

### Kyverno (Native Kubernetes Alternative)

YAML-based policies without Rego learning curve.

```yaml
apiVersion: kyverno.io/v1
kind: ClusterPolicy
metadata:
  name: require-ro-rootfs
spec:
  validationFailureAction: enforce
  rules:
  - name: check-read-only-root-fs
    match:
      resources:
        kinds:
        - Pod
    validate:
      message: "Root filesystem must be read-only"
      pattern:
        spec:
          containers:
          - securityContext:
              readOnlyRootFilesystem: true
```

**Image Verification:**
```yaml
apiVersion: kyverno.io/v1
kind: ClusterPolicy
metadata:
  name: verify-image-signatures
spec:
  validationFailureAction: enforce
  rules:
  - name: verify-signature
    match:
      resources:
        kinds:
        - Pod
    verifyImages:
    - imageReferences:
      - "company.registry.com/*"
      attestors:
      - entries:
        - keys:
            publicKeys: |
              -----BEGIN PUBLIC KEY-----
              MFkwEwYHKoZIzj0CAQYIKoZIzj0DAQcDQgAE...
              -----END PUBLIC KEY-----
```

### ImagePolicyWebhook (Native)

Built-in admission controller for image policy enforcement.

```yaml
# /etc/kubernetes/admission-config.yaml
apiVersion: apiserver.config.k8s.io/v1
kind: AdmissionConfiguration
plugins:
- name: ImagePolicyWebhook
  path: imagepolicy.json
```

**Image Policy Config:**
```json
{
  "imagePolicy": {
    "kubeConfigFile": "/etc/kubernetes/image-policy.kubeconfig",
    "allowTTL": 50,
    "denyTTL": 50,
    "retryBackoff": 500,
    "defaultAllow": false
  }
}
```

## 18.7 Security Contexts

Security contexts define privilege and access control settings for Pods or containers, implementing the security controls introduced in Chapter 10 at the Kubernetes orchestration layer.

### Pod-Level Security Context

Applied to all containers in the Pod:

```yaml
apiVersion: v1
kind: Pod
spec:
  securityContext:
    runAsNonRoot: true
    runAsUser: 1000
    runAsGroup: 1000
    fsGroup: 2000
    fsGroupChangePolicy: "OnRootMismatch"  # Only chmod on first access
    seccompProfile:
      type: RuntimeDefault  # Or Localhost with specific profile
    sysctls:
    - name: net.ipv4.ip_unprivileged_port_start
      value: "80"
```

### Container-Level Security Context

Overrides Pod-level settings:

```yaml
spec:
  containers:
  - name: app
    securityContext:
      allowPrivilegeEscalation: false
      readOnlyRootFilesystem: true
      capabilities:
        drop:
        - ALL
        add:
        - NET_BIND_SERVICE  # Only if binding privileged ports
      seLinuxOptions:
        level: "s0:c123,c456"
      seccompProfile:
        type: Localhost
        localhostProfile: profiles/custom.json
```

### AppArmor Integration

```yaml
apiVersion: v1
kind: Pod
metadata:
  annotations:
    container.apparmor.security.beta.kubernetes.io/nginx: localhost/nginx-profile
spec:
  containers:
  - name: nginx
    image: nginx:alpine
```

### Seccomp Profiles

**RuntimeDefault:**
Uses container runtime default seccomp profile (moderate restrictions).

**Custom Profile:**
```yaml
securityContext:
  seccompProfile:
    type: Localhost
    localhostProfile: profiles/strict.json
```

Profile must exist on node at `/var/lib/kubelet/seccomp/strict.json`.

## 18.8 Compliance and Auditing

Meeting regulatory requirements (SOC 2, PCI DSS, HIPAA, FedRAMP) requires comprehensive audit trails and continuous monitoring.

### API Server Audit Logging

Enable audit logging to track all API requests:

```yaml
# /etc/kubernetes/audit-policy.yaml
apiVersion: audit.k8s.io/v1
kind: Policy
rules:
- level: None
  users: ["system:kube-proxy", "system:kubelet"]
  verbs: ["watch"]
  
- level: Metadata
  resources:
  - group: ""
    resources: ["secrets", "configmaps"]
    
- level: RequestResponse
  resources:
  - group: "rbac.authorization.k8s.io"
    resources: ["clusterroles", "clusterrolebindings", "roles", "rolebindings"]
    
- level: RequestResponse
  verbs: ["create", "update", "delete", "patch"]
```

**API Server Configuration:**
```yaml
apiVersion: v1
kind: Pod
metadata:
  name: kube-apiserver
spec:
  containers:
  - name: kube-apiserver
    command:
    - kube-apiserver
    - --audit-policy-file=/etc/kubernetes/audit-policy.yaml
    - --audit-log-path=/var/log/audit/audit.log
    - --audit-log-maxage=30
    - --audit-log-maxbackup=10
    - --audit-log-maxsize=100
```

### Falco (Runtime Security)

Detect anomalous behavior at runtime using Falco rules.

**Installation:**
```bash
helm repo add falcosecurity https://falcosecurity.github.io/charts
helm install falco falcosecurity/falco \
  --set falcosidekick.enabled=true \
  --set falcosidekick.webhook.address=http://alert-manager:9093
```

**Rule Example:**
```yaml
- rule: Terminal Shell in Container
  desc: Detect shell execution inside containers
  condition: >
    spawned_process and
    container and
    shell_procs and
    (not allowed_shell_procs)
  output: >
    Shell executed in container
    (user=%user.name container=%container.name cmdline=%proc.cmdline)
  priority: WARNING
```

### CIS Benchmark Scanning

Use kube-bench to verify CIS Kubernetes Benchmark compliance:

```bash
kubectl apply -f https://raw.githubusercontent.com/aquasecurity/kube-bench/main/job.yaml
kubectl logs job/kube-bench -n default
```

**Remediation:**
Address FAIL results by modifying API server flags, kubelet configuration, or RBAC policies.

### Continuous Compliance

**OPA/Gatekeeper Constraints for Compliance:**
```yaml
apiVersion: constraints.gatekeeper.sh/v1beta1
kind: K8sRequiredProbes
metadata:
  name: must-have-probes
spec:
  match:
    kinds:
      - apiGroups: [""]
        kinds: ["Pod"]
  parameters:
    probes:
      - "readinessProbe"
      - "livenessProbe"
```

---

## Chapter Summary and Preview

In this chapter, we implemented comprehensive security controls for Kubernetes clusters. RBAC establishes role-based access with least privilege principles, distinguishing between human users and ServiceAccounts while utilizing token projection for short-lived credentials. Pod Security Standards enforce three-tiered security levels (Privileged, Baseline, Restricted) preventing privilege escalation and insecure configurations. Network Policies implement zero-trust micro-segmentation with default-deny postures. Secrets management integrates external providers via External Secrets Operator or Vault Agent Injector, eliminating long-lived credentials in etcd. Admission controllers (OPA Gatekeeper, Kyverno) enforce policy-as-code preventing deployment of vulnerable images or non-compliant configurations. Security contexts harden runtime environments with seccomp, AppArmor, and capability restrictions. Finally, audit logging and Falco runtime detection ensure compliance visibility and threat detection.

**Key Takeaways:**
- Never use the default ServiceAccount; explicitly specify ServiceAccounts with minimal RBAC and disable automount when API access unnecessary.
- Enforce Pod Security Standards at the Restricted level in production namespaces, using Baseline only for legacy compatibility.
- Implement default-deny NetworkPolicies in all namespaces; explicitly allow only required communication paths.
- Use external secret management (Vault, AWS Secrets Manager) rather than native Kubernetes Secrets for sensitive credentials; if using native Secrets, enable encryption at rest.
- Deploy admission controllers to enforce image signing verification, required labels, and security context policies before objects reach the API server.
- Enable comprehensive API audit logging with appropriate retention for compliance investigations.

**Next Chapter Preview:**
Chapter 19: GitOps with ArgoCD introduces the modern continuous delivery paradigm for Kubernetes. You will implement declarative application delivery where Git repositories serve as the single source of truth for infrastructure and application state. We will deploy ArgoCD to automatically synchronize cluster state with Git commits, implement Application and ApplicationSet resources for multi-cluster deployments, configure automated synchronization with pruning and self-healing, and establish Progressive Delivery patterns using Argo Rollouts for canary and blue/green deployments. This operational model transforms the security-hardened cluster from this chapter into a self-managing, version-controlled infrastructure platform.